In [0]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [0]:
from google.colab import files
uploades = files.upload()

Saving modified_data.csv to modified_data.csv


In [0]:
data = pd.read_csv('modified_data.csv',header=0,na_filter=True,)

In [0]:
pre_data = data.drop(columns=['Id','SalePrice'],axis=1)
all_features = list(pre_data.columns)
len(all_features)

75

In [0]:
categorical_data=data.select_dtypes(include=['object']).copy()
categorical_features=list(categorical_data.columns)
#categorical_data.head()
categorical_features

In [0]:
nominal=['MSZoning', 'LandContour', 'LotConfig', 'Neighborhood', 'RoofStyle', 'Functional', 'SaleType']#['LOC','FUR','WATER','HK_SER']
ordinal=list(set(categorical_features)-set(nominal))#['AMB','PROX_SCH','PROX_ROAD','PROX_MALL']
numerical=list(set(all_features)-set(categorical_features))#['SIZE']
target=['SalePrice']

for feature in categorical_features:
  categorical_data[feature]=categorical_data[feature].astype('category')
for feature in ordinal:
  categorical_data[feature]=categorical_data[feature].cat.codes
categorical_data.head()

In [0]:
data_copy=categorical_data.copy()

data_copy = pd.concat([pd.get_dummies(data_copy[nominal]),data_copy],axis=1) 
data_copy.drop(nominal,axis=1,inplace=True)
#data_copy
new_data = pd.concat([data_copy,data[numerical],data[target]],axis=1)
new_data

In [0]:
all_features = list(new_data.columns)
all_features
X = new_data.loc[:,all_features[0]:all_features[-2]].values
y = new_data.loc[:,[all_features[-1]]].values
#X

#X = new_data.drop(target,1)
#y = new_data[target]

In [0]:
X = StandardScaler().fit_transform(np.nan_to_num(X))
X

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [0]:
X_train.shape

In [0]:
regressor=Lasso()
#kfold = KFold(n_splits=3)
regressor.fit(X_train,y_train)

#results = cross_val_score(regressor,X_train,y_train,cv=kfold)
#results.mean()

In [0]:
regressor.score(X_test,y_test)

0.7508093518897022

Cross Validation

---



In [0]:
kfold = KFold(n_splits=3)
scores = cross_val_score(regressor,X_train,y_train,cv=kfold)
scores

In [0]:
scores.mean()

Grid Search Hyperparameter Tuning

---



In [0]:
model = Lasso()
lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03],'max_iter':[1000,1100,2000],'tol':[0.001,0.0001,0.000001]}
#param_grid = dict(alpha = lasso_params)
grid = GridSearchCV(model,param_grid=lasso_params,cv=2,n_jobs=-1)
#grid.fit(X,y)
results = grid.fit(X_train,y_train)
#print(results.best_score_)
#print("Best Params are ", results.best_params_)
results.score(X_test,y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79429144239.65381, tolerance: 6450040898.708485
  positive)


0.75059994649858

Random Search Hyperparameter Tuning

---



In [0]:
random = RandomizedSearchCV(model,param_distributions=lasso_params,cv=3,n_jobs=-1)
results = random.fit(X_train,y_train)
print(results.best_score_)
print("Best Params are ", results.best_params_)
results.score(X_test,y_test)

0.832989580812621
Best Params are  {'tol': 1e-06, 'max_iter': 1000, 'alpha': 0.03}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79429144239.65381, tolerance: 6450040.898708484
  positive)


0.75059994649858

In [0]:
predictions = regressor.predict(X_test)
mean_squared_error(y_test,predictions)
#accuracy_score(y_test,predictions)

81.13716726282684

In [0]:
pca = PCA(n_components=2)
pc_s = pca.fit_transform(X)
pca_data = pd.DataFrame(pc_s,columns=['PC1','PC2'])
final_data = pd.concat([pca_data,data[target]],axis=1)
final_data

In [0]:
pca.explained_variance_ratio_

array([0.2446499 , 0.18305979])

In [0]:
#regressor=LassoCV()
X = final_data.drop("PRICE",1)
y = final_data["PRICE"]
regressor=LassoCV()
regressor.fit(X,y)

In [0]:
regressor.coef_

array([1.67696472, 4.05817657])

In [0]:
regressor.score(X,y)

0.3675453513605128

In [0]:
kfold = KFold(n_splits=3)

In [0]:
results = cross_val_score(regressor,X,y,cv=kfold)
results.mean()

-0.3601711600231899